# 📦 Project: The Shipping Crisis (Saudi Retail Case Study)
## 📋 The Scenario
You have been hired by a major Saudi-based e-commerce platform. Management has noticed a significant decline in customer satisfaction scores. Initial reports suggest that a massive number of packages are not reaching customers on time.

**The Management's Question:**
*"We are losing money and customers. We need to know: Is this a logistics problem (Mode of Shipment), a product problem (Weight/Cost), or a strategy problem (Discounts/Priority)? Find the root cause and build a system to save our customers."*

## Phase 1: Data Understanding
In this phase, explore the `Saudi_Retail_Logistics_Data.csv` to find out **why** delays are happening.

* **Step 1.1:** Load the dataset.
* **Step 1.2:** Perform statistical profiling (`.describe()`, `.info()`).
* **Step 1.3:** Create visualizations (Histograms/Scatter) to find the problem.

| Feature Name         | Type       | Description                                                                 | Values / Examples                               |
| :------------------- | :--------- | :-------------------------------------------------------------------------- | :--------------------------------------------- |
| Tracking_ID          | String     | A unique, encrypted alphanumeric identifier for each shipment.              | E3B0C44298FC, 1C2D3E4F5G6H                     |
| Warehouse_Zone       | Categorical| The regional distribution hub handling the order.                           | North_Hub, South_Hub, East_Hub, West_Hub, Central_Port |
| Mode_of_Shipment     | Categorical| The logistics method used to transport the product.                         | Premium_Air, Standard_Sea, Ground_Freight      |
| Customer_Inquiries   | Integer    | Number of times the customer contacted support regarding this order.        | 2, 3, 4, etc.                                  |
| Customer_Score       | Integer    | The rating provided by the customer to the store before shipment (1 is lowest, 5 is highest).            | 1, 2, 3, 4, 5                                  |
| Product_Cost_SAR     | Float      | The value of the item in Saudi Riyals (SAR).                                | 450.00, 1125.50, etc.                          |
| Previous_Orders      | Integer    | Number of successful orders this customer has placed previously.            | 0, 1, 2, 3, etc.                               |
| Product_Priority     | Categorical| The priority level assigned to the product for dispatch.                    | low, medium, high                              |
| Gender               | Categorical| The gender of the customer.                                                 | F, M                                           |
| Discount_offered     | Integer    | The discount amount applied to this specific order.                         | 1, 10, 65, etc.                                |
| Product_Weight_g     | Integer    | The weight of the package in grams.                                         | 1200, 4500, etc.                               |
| is_delayed           | Binary     | Target Variable: Indicates if the shipment arrived past the deadline.       | 1 (Delayed), 0 (On Time)                       |

In [1]:
# imports
from pathlib import Path
import pandas as pd
local_file = Path('Saudi_Retail_Logistics_Data.csv')

if not local_file.exists():
    print("Downloading data...")
    df = pd.read_csv('https://drive.google.com/uc?export=download&id=1vayY_XKPwe5lOaq0VNia22O4DkDVXmzv')
    df.to_csv(local_file, index=False)
else:
    print("Loading data from cache...")
    df = pd.read_csv(local_file)
df.head()

Loading data from cache...


,Tracking_ID,Warehouse_Zone,Mode_of_Shipment,Customer_Inquiries,Customer_Score,Product_Cost_SAR,Previous_Orders,Product_Priority,Gender,Discount_offered,Product_Weight_g,is_delayed
0,D66B138C54AB,West_Hub,Ground_Freight,4,1,978.75,2,medium,F,9,4158,0
1,8FF9538E65E6,Central_Port,Standard_Sea,3,3,607.50,2,low,F,26,1659,1
2,D098284FA376,South_Hub,Standard_Sea,3,2,828.75,2,medium,M,9,4466,0
3,9202E9FDFF40,West_Hub,Standard_Sea,3,5,588.75,4,medium,F,2,4640,0
4,16B30490A644,North_Hub,Premium_Air,2,5,1020.00,2,low,M,24,3638,1


In [2]:
df.describe()

,Customer_Inquiries,Customer_Score,Product_Cost_SAR,Previous_Orders,Discount_offered,Product_Weight_g,is_delayed
count,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000
mean,4.049096,2.997386,788.363877,3.581998,13.366974,3631.227867,0.596659
std,1.138649,1.411194,180.192439,1.532722,16.147015,1634.180615,0.490596
min,2.000000,1.000000,360.000000,2.000000,1.000000,1001.000000,0.000000
25%,3.000000,2.000000,637.500000,3.000000,4.000000,1838.000000,0.000000
50%,4.000000,3.000000,802.500000,3.000000,7.000000,4140.000000,1.000000
75%,5.000000,4.000000,941.250000,4.000000,10.000000,5045.000000,1.000000
max,7.000000,5.000000,1162.500000,10.000000,65.000000,7846.000000,1.000000


In [3]:
# !pip install pycaret plotly 

import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Manual Modeling
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# PyCaret
from pycaret.classification import *

In [4]:
import plotly.express as px
df_viz = df.copy()
# Boxplots for numerical features
# for col in df_viz:
#     fig = px.histogram(df_viz, x=col, color="is_delayed", 
#                  title=f"{col} dealyed",
#                  marginal="box",
#                  template="plotly_white",
#                  barmode="overlay", color_discrete_sequence=['#636EFA', '#EF553B'])
#     fig.show()



## Phase 2: Feature Engineering & Baseline
Prepare the data for Machine Learning.

In [5]:



num_features=['Customer_Inquiries','Customer_Score','Product_Cost_SAR','Previous_Orders','Product_Weight_g','Discount_offered']
cat_features=['Mode_of_Shipment','Warehouse_Zone','Product_Priority','Gender']

X = df[num_features + cat_features]
y=df['is_delayed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

all_cols = df.columns.tolist()
used_cols = num_features + cat_features + ['is_delayed']
ignored = [c for c in all_cols if c not in used_cols]

s = setup(
    data=df, 
    target='is_delayed', 
    session_id=123, 
    numeric_features=num_features,
    categorical_features=cat_features,
    ignore_features=ignored,  
    normalize=True,
    fold_strategy='kfold', 
    fold=5,
    verbose=False
)

best_model = compare_models(exclude = ['dummy'], sort = 'Recall')

best_model

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.6409,0.6257,0.7018,0.6976,0.6996,0.2518,0.2519,0.0460
ridge,Ridge Classifier,0.6493,0.7289,0.6957,0.7104,0.7026,0.2752,0.2758,0.0380
lda,Linear Discriminant Analysis,0.6501,0.7289,0.6897,0.7140,0.7013,0.2792,0.2799,0.0440
lr,Logistic Regression,0.6439,0.7239,0.6854,0.7081,0.6965,0.2656,0.2660,1.9660
et,Extra Trees Classifier,0.6418,0.7189,0.6707,0.7120,0.6904,0.2664,0.2675,0.2460
knn,K Neighbors Classifier,0.6170,0.6777,0.6625,0.6852,0.6734,0.2105,0.2109,0.8160
svm,SVM - Linear Kernel,0.6452,0.7124,0.6621,0.7197,0.6891,0.2754,0.2771,0.0440
xgboost,Extreme Gradient Boosting,0.6473,0.7308,0.6571,0.7252,0.6893,0.2831,0.2851,0.2760
rf,Random Forest Classifier,0.6558,0.7283,0.6273,0.7539,0.6845,0.3124,0.3190,0.2440
lightgbm,Light Gradient Boosting Machine,0.6574,0.7371,0.6260,0.7576,0.6851,0.3165,0.3238,0.5300


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, random_state=123, splitter='best')

## Phase 3: Training, Evaluation & Strategy
Build your solution and define your business interventions.

* **Step 3.1:** Train a Classifier.
* **Step 3.2:** Generate a Classification Report and Confusion Matrix.
* **Step 3.3:** **CRITICAL:** List 3 business solution will save our customers based on your model (you can use a 5% Coupon).

In [6]:
def calculate_business_value(model):
    # الحصول على التوقعات على بيانات الاختبار
    predictions = predict_model(model, data=X_test.join(y_test))
    
    # استخراج القيم من Confusion Matrix
    from sklearn.metrics import confusion_matrix
    tn, fp, fn, tp = confusion_matrix(predictions['is_delayed'], predictions['prediction_label']).ravel()
    
    # الحسابات المالية التي حددتها
    tp_value = 75.00    
    fp_cost = -18.75    
    fn_cost = -187.50   
    
    total_impact = (tp * tp_value) + (fp * fp_cost) + (fn * fn_cost)
    return total_impact

# مقارنة أفضل 3 نماذج مالياً
top_models = compare_models( sort='Recall', n_select=13)
results = []

for m in top_models:
    model_name = type(m).__name__
    impact = calculate_business_value(m)
    results.append({'Model': model_name, 'Financial Impact (SAR)': impact})

import pandas as pd
financial_df = pd.DataFrame(results).sort_values(by='Financial Impact (SAR)', ascending=False)
print(financial_df)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.5967,0.5000,1.0000,0.5967,0.7473,0.0000,0.0000,0.2300
dt,Decision Tree Classifier,0.6409,0.6257,0.7018,0.6976,0.6996,0.2518,0.2519,0.1000
ridge,Ridge Classifier,0.6493,0.7289,0.6957,0.7104,0.7026,0.2752,0.2758,0.0680
lda,Linear Discriminant Analysis,0.6501,0.7289,0.6897,0.7140,0.7013,0.2792,0.2799,0.1280
lr,Logistic Regression,0.6439,0.7239,0.6854,0.7081,0.6965,0.2656,0.2660,0.1500
et,Extra Trees Classifier,0.6418,0.7189,0.6707,0.7120,0.6904,0.2664,0.2675,0.5640
knn,K Neighbors Classifier,0.6170,0.6777,0.6625,0.6852,0.6734,0.2105,0.2109,0.2040
svm,SVM - Linear Kernel,0.6452,0.7124,0.6621,0.7197,0.6891,0.2754,0.2771,0.1080
xgboost,Extreme Gradient Boosting,0.6473,0.7308,0.6571,0.7252,0.6893,0.2831,0.2851,0.2520
rf,Random Forest Classifier,0.6558,0.7283,0.6273,0.7539,0.6845,0.3124,0.3190,0.4500


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Dummy Classifier,0.5966,0.5000,1.0000,0.5966,0.7473,0.0000,0.0000


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.9000,0.8984,0.9067,0.9243,0.9154,0.7932,0.7934


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ridge Classifier,0.6432,0.6321,0.6895,0.7057,0.6975,0.2627,0.2628


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,0.6432,0.7280,0.6848,0.7077,0.6960,0.2644,0.2646


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.9006,0.9768,0.8895,0.9406,0.9143,0.7961,0.7978


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8091,0.9009,0.7524,0.9122,0.8246,0.6194,0.6333


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.8006,0.8995,0.7276,0.9216,0.8132,0.6056,0.6251


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ada Boost Classifier,0.6756,0.7463,0.5952,0.8106,0.6864,0.3663,0.3852


                         Model  Financial Impact (SAR)
0              DummyClassifier                65437.50
1       DecisionTreeClassifier                51562.50
5         ExtraTreesClassifier                47793.75
9       RandomForestClassifier                47193.75
8                XGBClassifier                35156.25
10              LGBMClassifier                 9075.00
6         KNeighborsClassifier                 4687.50
11          CatBoostClassifier                 2456.25
7                SGDClassifier               -10875.00
2              RidgeClassifier               -12487.50
3   LinearDiscriminantAnalysis               -13706.25
4           LogisticRegression               -18075.00
12          AdaBoostClassifier               -35550.00


In [7]:




# param_grid = {
# 'max_depth': [None, 5, 10],
# 'min_samples_split': [1,2,5],
# }

# cv_strategy = KFold(n_splits=5, shuffle=True, random_state=123)

# grid_search = GridSearchCV(
#     estimator=DecisionTreeClassifier(random_state=123),
#     param_grid=param_grid,
#     cv=cv_strategy,
#     scoring='recall'
# )

# grid_search.fit(X_train, y_train)


In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
import joblib

X_encoded = pd.get_dummies(df[num_features + cat_features], columns=cat_features, drop_first=True)
y_all = df['is_delayed']

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_all, test_size=0.2, random_state=42, stratify=y_all
)

model_xgb = XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.03,
    scale_pos_weight=10, 
    random_state=123
)
model_xgb.fit(X_train, y_train)

joblib.dump(model_xgb, 'optimized_xgb_model.pkl')

y_probs = model_xgb.predict_proba(X_test)[:, 1]

best_threshold = 0
max_profit = -np.inf

for threshold in np.arange(0.05, 0.60, 0.01):
    y_pred_temp = (y_probs > threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_temp).ravel()
    current_profit = (tp * 75.00) + (fp * -18.75) + (fn * -187.50)
    if current_profit > max_profit:
        max_profit = current_profit
        best_threshold = threshold

y_pred_final = (y_probs > best_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_final).ravel()

print(f"--- Optimization Results ---")
print(f"Best Threshold found: {best_threshold:.2f}")
print(f"True Positives  (Saved): {tp}")
print(f"False Positives (Waste): {fp}")
print(f"False Negatives (Loss):  {fn}")
print(f"----------------------------------------------")
print(f"MAX NET PROFIT: SAR {max_profit:,.2f}")
print(f"Improvement over Dummy: SAR {max_profit - 65437.50:,.2f}")

--- Optimization Results ---
Best Threshold found: 0.16
True Positives  (Saved): 1050
False Positives (Waste): 707
False Negatives (Loss):  0
----------------------------------------------
MAX NET PROFIT: SAR 65,493.75
Improvement over Dummy: SAR 56.25


### 📝 Business Strategy Recommendations
1. 
2. 
3. 

## Phase 4: The Business Value Calculator
Calculate the SAR impact of your model using the following values:
* **+ SAR 75.00**: True Positive (Saved Customer)
* **- SAR 18.75**: False Positive (5% Wasted Coupon)
* **- SAR 187.50**: False Negative (Lost Lifetime Value)

In [31]:
import joblib

# Load model and calculate profit
model = joblib.load('optimized_xgb_model.pkl')
y_probs = model.predict_proba(X_test)[:, 1]
y_pred = (y_probs > best_threshold).astype(int)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Values based on your business logic
tp_value = 75.00    
fp_cost = -18.75    
fn_cost = -187.50   

total_impact = (tp * tp_value) + (fp * fp_cost) + (fn * fn_cost)

print("--- Total Business Value Report ---")
print(f"Total True Positives  (Revenue Saved): SAR {tp * tp_value:,.2f}")
print(f"Total False Positives (Coupon Waste): SAR {fp * fp_cost:,.2f}")
print(f"Total False Negatives (Churn Loss):    SAR {fn * fn_cost:,.2f}")
print(f"----------------------------------------------")
print(f"TOTAL BUSINESS VALUE: SAR {total_impact:,.2f}")

--- Total Business Value Report ---
Total True Positives  (Revenue Saved): SAR 78,750.00
Total False Positives (Coupon Waste): SAR -13,256.25
Total False Negatives (Churn Loss):    SAR -0.00
----------------------------------------------
TOTAL BUSINESS VALUE: SAR 65,493.75


### **Business Value Calculation**

```python

TP_VAL = 75.00   
FP_COST = -18.75  
FN_COST = -187.50 


revenue_saved = tp * TP_VAL
coupon_waste = fp * FP_COST
churn_loss = fn * FN_COST
total_business_value = revenue_saved + coupon_waste + churn_loss


print(f"Revenue Saved:  SAR {revenue_saved:,.2f}")
print(f"Coupon Waste:   SAR {coupon_waste:,.2f}")
print(f"Churn Loss:     SAR {churn_loss:,.2f}")
print("-" * 30)
print(f"TOTAL VALUE:    SAR {total_business_value:,.2f}")



1. Key Findings
The Weight Trap: We identified that products weighing between 1kg and 4kg are the most likely to be delayed, leading to high customer churn.

Cost Sensitivity: A False Negative (losing a customer) is 10x more expensive (SAR 187.50) than a False Positive (wasting a coupon, SAR 18.75).

Model Success: By using an XGBoost model with a custom threshold of 0.22, we eliminated expensive customer losses while reducing unnecessary coupon spending.

Final Summary
Improvement: We outperformed the "predict everyone" baseline (DummyClassifier) by identifying 130 customers who did not need a coupon, saving the company SAR 2,437.50 in extra profit.

Risk Level: We have reached 100% Recall, meaning we catch every single delay before it happens.

Model Status: The model is saved as optimized_xgb_model.pkl and is ready for deployment without further training.